# Série temporal do Covid19 em Portugal

In [138]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, cross_validate, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from xgboost import XGBRegressor

In [139]:
# importando  Dataset
df_pt = pd.read_csv("https://raw.githubusercontent.com/dssg-pt/covid19pt-data/master/data.csv")

In [140]:
# Redefinindo o tipo do campo de data
df_pt['data'] = pd.to_datetime(df_pt['data'])
df_pt['data_dados'] = pd.to_datetime(df_pt['data_dados'])

In [141]:
df_pt.head()

,data,data_dados,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_estrangeiro,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada,confirmados_0_9_f,confirmados_0_9_m,confirmados_10_19_f,confirmados_10_19_m,confirmados_20_29_f,confirmados_20_29_m,confirmados_30_39_f,confirmados_30_39_m,confirmados_40_49_f,confirmados_40_49_m,confirmados_50_59_f,confirmados_50_59_m,confirmados_60_69_f,confirmados_60_69_m,confirmados_70_79_f,confirmados_70_79_m,confirmados_80_plus_f,confirmados_80_plus_m,...,obitos_acores,obitos_madeira,obitos_estrangeiro,recuperados_arsnorte,recuperados_arscentro,recuperados_arslvt,recuperados_arsalentejo,recuperados_arsalgarve,recuperados_acores,recuperados_madeira,recuperados_estrangeiro,obitos_0_9_f,obitos_0_9_m,obitos_10_19_f,obitos_10_19_m,obitos_20_29_f,obitos_20_29_m,obitos_30_39_f,obitos_30_39_m,obitos_40_49_f,obitos_40_49_m,obitos_50_59_f,obitos_50_59_m,obitos_60_69_f,obitos_60_69_m,obitos_70_79_f,obitos_70_79_m,obitos_80_plus_f,obitos_80_plus_m,obitos_f,obitos_m,confirmados_desconhecidos_m,confirmados_desconhecidos_f,ativos,internados_enfermaria,confirmados_desconhecidos,incidencia_nacional,incidencia_continente,rt_nacional,rt_continente
0,2020-02-26,2020-02-26,0,0,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,NaN,25.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-27,2020-02-27,0,0,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,NaN,51.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-28,2020-02-28,0,0,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,NaN,59.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-29,2020-02-29,0,0,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,NaN,70.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-03,2020-01-03,0,0,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,NaN,85.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
df_pt.tail(15)

,data,data_dados,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_estrangeiro,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada,confirmados_0_9_f,confirmados_0_9_m,confirmados_10_19_f,confirmados_10_19_m,confirmados_20_29_f,confirmados_20_29_m,confirmados_30_39_f,confirmados_30_39_m,confirmados_40_49_f,confirmados_40_49_m,confirmados_50_59_f,confirmados_50_59_m,confirmados_60_69_f,confirmados_60_69_m,confirmados_70_79_f,confirmados_70_79_m,confirmados_80_plus_f,confirmados_80_plus_m,...,obitos_acores,obitos_madeira,obitos_estrangeiro,recuperados_arsnorte,recuperados_arscentro,recuperados_arslvt,recuperados_arsalentejo,recuperados_arsalgarve,recuperados_acores,recuperados_madeira,recuperados_estrangeiro,obitos_0_9_f,obitos_0_9_m,obitos_10_19_f,obitos_10_19_m,obitos_20_29_f,obitos_20_29_m,obitos_30_39_f,obitos_30_39_m,obitos_40_49_f,obitos_40_49_m,obitos_50_59_f,obitos_50_59_m,obitos_60_69_f,obitos_60_69_m,obitos_70_79_f,obitos_70_79_m,obitos_80_plus_f,obitos_80_plus_m,obitos_f,obitos_m,confirmados_desconhecidos_m,confirmados_desconhecidos_f,ativos,internados_enfermaria,confirmados_desconhecidos,incidencia_nacional,incidencia_continente,rt_nacional,rt_continente
707,2022-02-02,2022-02-02,2745383,1060869,389740,996552,91089,105676,35954,65503,NaN,54693,2112346,20024,2442.00,149.00,NaN,NaN,645697.00,NaN,NaN,NaN,135259.00,141652.00,174379.00,176221.00,219766.00,207373.00,234773.00,200515.00,264593.00,215266.00,185387.00,153523.00,108729.00,96343.00,63452.00,55863.00,71702.00,38073.00,...,62,164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,2.00,2.00,1.00,8.00,10.00,22.00,32.00,82.00,122.00,179.00,415.00,567.00,1271.00,1642.00,2699.00,6982.00,5987.00,9485.00,10539.00,NaN,NaN,613013.00,2293.00,2514.00,7081.70,7111.80,1.09,1.10
708,2022-03-02,2022-03-02,2795830,1081148,399195,1010845,93361,107779,37465,66037,NaN,50447,2133640,20077,2440.00,155.00,NaN,NaN,653062.00,NaN,NaN,NaN,138466.00,144948.00,178615.00,180540.00,223655.00,210641.00,239648.00,204353.00,269803.00,219382.00,187898.00,155401.00,110304.00,97524.00,64365.00,56700.00,72511.00,38524.00,...,64,167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,2.00,2.00,1.00,8.00,10.00,22.00,32.00,82.00,123.00,179.00,415.00,568.00,1275.00,1646.00,2706.00,7001.00,6004.00,9509.00,10568.00,NaN,NaN,642113.00,2285.00,2552.00,7081.70,7111.80,1.09,1.10
709,2022-04-02,2022-04-02,2843029,1099251,408393,1024826,95313,109979,38656,66611,NaN,47199,2180109,20127,2445.00,174.00,NaN,NaN,660347.00,NaN,NaN,NaN,141461.00,148026.00,182564.00,184572.00,227277.00,213778.00,244102.00,207879.00,274544.00,223037.00,190289.00,157201.00,111792.00,98697.00,65373.00,57481.00,73366.00,38993.00,...,66,170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,2.00,2.00,1.00,8.00,10.00,22.00,32.00,83.00,123.00,180.00,417.00,571.00,1278.00,1648.00,2713.00,7018.00,6018.00,9533.00,10594.00,NaN,NaN,642793.00,2271.00,2597.00,7163.70,7207.00,1.05,1.05
710,2022-05-02,2022-05-02,2884540,1115235,416658,1036682,97119,111877,39753,67216,NaN,41511,2226548,20171,2409.00,169.00,NaN,NaN,665706.00,NaN,NaN,NaN,143978.00,150638.00,186165.00,188207.00,230350.00,216556.00,247909.00,210867.00,278686.00,226131.00,192513.00,158849.00,113158.00,99782.00,66199.00,58264.00,74198.00,39478.00,...,68,170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,2.00,2.00,1.00,8.00,10.00,22.00,32.00,83.00,125.00,180.00,419.00,571.00,1279.00,1653.00,2720.00,7038.00,6025.00,9558.00,10613.00,NaN,NaN,637821.00,2240.00,2612.00,7163.70,7207.00,1.05,1.05
711,2022-06-02,2022-06-02,2915971,1126462,422504,1046521,98358,113514,40966,67646,NaN,31431,2266939,20222,2511.00,180.00,NaN,NaN,664442.00,NaN,NaN,NaN,146066.00,152863.00,188983.00,191057.00,232724.00,218508.00,250783.00,213173.00,281765.00,228488.00,194087.00,160072.00,114107.00,100528.00,66818.00,58795.00,74742.00,39788.00,...,70,170

In [143]:
df_pt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 93 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   data                               722 non-null    datetime64[ns]
 1   data_dados                         722 non-null    datetime64[ns]
 2   confirmados                        722 non-null    int64         
 3   confirmados_arsnorte               722 non-null    int64         
 4   confirmados_arscentro              722 non-null    int64         
 5   confirmados_arslvt                 722 non-null    int64         
 6   confirmados_arsalentejo            722 non-null    int64         
 7   confirmados_arsalgarve             722 non-null    int64         
 8   confirmados_acores                 722 non-null    int64         
 9   confirmados_madeira                722 non-null    int64         
 10  confirmados_estrangeiro            16 

In [144]:
round(df_pt.describe(), 2)

,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_estrangeiro,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada,confirmados_0_9_f,confirmados_0_9_m,confirmados_10_19_f,confirmados_10_19_m,confirmados_20_29_f,confirmados_20_29_m,confirmados_30_39_f,confirmados_30_39_m,confirmados_40_49_f,confirmados_40_49_m,confirmados_50_59_f,confirmados_50_59_m,confirmados_60_69_f,confirmados_60_69_m,confirmados_70_79_f,confirmados_70_79_m,confirmados_80_plus_f,confirmados_80_plus_m,sintomas_tosse,sintomas_febre,...,obitos_acores,obitos_madeira,obitos_estrangeiro,recuperados_arsnorte,recuperados_arscentro,recuperados_arslvt,recuperados_arsalentejo,recuperados_arsalgarve,recuperados_acores,recuperados_madeira,recuperados_estrangeiro,obitos_0_9_f,obitos_0_9_m,obitos_10_19_f,obitos_10_19_m,obitos_20_29_f,obitos_20_29_m,obitos_30_39_f,obitos_30_39_m,obitos_40_49_f,obitos_40_49_m,obitos_50_59_f,obitos_50_59_m,obitos_60_69_f,obitos_60_69_m,obitos_70_79_f,obitos_70_79_m,obitos_80_plus_f,obitos_80_plus_m,obitos_f,obitos_m,confirmados_desconhecidos_m,confirmados_desconhecidos_f,ativos,internados_enfermaria,confirmados_desconhecidos,incidencia_nacional,incidencia_continente,rt_nacional,rt_continente
count,722.00,722.00,722.00,722.00,722.00,722.00,722.00,722.00,16.00,722.00,722.00,722.00,714.00,705.00,164.00,173.00,715.00,155.00,15.00,167.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,715.00,167.00,167.00,...,722.00,722.00,31.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,693.00,693.00,693.00,693.00,693.00,693.00,695.00,695.00,76.00,76.00,717.00,717.00,666.00,337.00,337.00,337.00,337.00
mean,655675.54,258966.14,89360.21,248914.50,22459.71,22063.64,5275.16,8635.96,9.12,4337.81,577384.22,10541.48,1237.37,194.46,2210.07,258703.39,77162.87,254759.04,12.20,640.63,21186.57,22108.82,34248.86,34079.04,53087.18,49346.39,52692.77,45748.48,60620.55,48853.75,51192.44,40777.63,32101.68,29283.66,20657.03,18352.95,30593.58,15384.70,0.44,0.34,...,27.48,45.23,0.00,10.80,9.10,13.90,0.00,0.00,0.00,0.00,0.00,0.79,0.82,0.71,0.61,3.42,4.80,12.26,14.63,41.68,63.36,90.41,218.01,299.77,683.82,872.00,1453.52,3901.13,3292.85,5220.36,5730.56,27.22,20.05,68222.29,1040.90,638.41,812.66,814.13,1.04,1.04
std,624213.94,236383.08,89038.15,233549.21,22117.66,25064.21,7596.40,13433.51,5.86,9993.36,536601.64,7747.09,1377.30,193.42,1387.71,157914.29,115857.30,123634.45,7.69,249.60,26986.86,28173.28,37678.74,37878.16,50643.32,48363.84,51322.20,44682.25,58335.51,47573.34,44737.61,36525.52,27459.74,24780.69,16950.79,15011.43,22703.10,11601.66,0.11,0.11,...,15.89,45.67,0.00,6.71,1.20,4.48,0.00,0.00,0.00,0.00,0.00,0.41,0.84,0.62,0.49,2.21,3.28,8.78,11.70,28.46,44.61,63.16,151.05,211.98,481.49,614.21,1030.51,2670.36,2285.82,3598.23,4008.62,14.59,9.23,112624.80,1193.66,1049.38,1694.62,1696.23,0.12,0.13
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,9.00,9.00,30.00,25.00,81.00,1746.00,4.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.10,0.15,...,0.00,0.00,0.00,3.00,7.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2.00,0.00,4.00,0.00,5.00,9.00,12.00,10.00,23.00,0.00,0.00,2.00,0.00,-152.00,50.30,47.50,0.76,0.75
25%,55768.00,19984.25,4671.00,28845.00,889.75,1035.25,202.50,140.25,5.00,388.00,40915.25,1802.00,406.25,70.00,1417.75,115158.00,25716.00,189274.00,6.00,707.00,1003.00,1141.00,1459.50,1309.50,4774.50,4190.50,4932.50,4485.00,5167.00,4269.00,4822.50,3695.00,3005.50,2701.50,2074.00,1848.50,4249.00,2055.50,0.36,0.28,...,15.00,0.00,0.00,3.00,8.00,11.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,1.00,3.00,1.00,10

## Ajustando as Features

### Excluindo features 

In [145]:
df_pt = df_pt.drop(['data_dados', 'confirmados_estrangeiro', 'cadeias_transmissao', 
                    'obitos_estrangeiro','recuperados_arscentro','recuperados_arslvt',
                    'recuperados_arsalentejo','recuperados_arsalgarve','recuperados_acores', 'recuperados_arsnorte',
                    'recuperados_madeira','recuperados_estrangeiro','obitos_arsnorte',
                    'obitos_arscentro','obitos_arslvt','obitos_arsalentejo','obitos_arsalgarve',
                    'obitos_acores','obitos_madeira','confirmados_arsnorte', 'confirmados_arscentro',
                    'confirmados_arslvt', 'confirmados_arsalentejo','confirmados_arsalgarve', 'confirmados_acores', 
                    'confirmados_madeira', 'transmissao_importada', 'confirmados_desconhecidos', 
                    'confirmados_desconhecidos_m', 'confirmados_desconhecidos_f'
                   ], axis=1)

In [146]:
df_pt.shape

(722, 63)

In [147]:
df_pt.columns

Index(['data', 'confirmados', 'confirmados_novos', 'recuperados', 'obitos',
       'internados', 'internados_uci', 'lab', 'suspeitos', 'vigilancia',
       'n_confirmados', 'confirmados_0_9_f', 'confirmados_0_9_m',
       'confirmados_10_19_f', 'confirmados_10_19_m', 'confirmados_20_29_f',
       'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m',
       'confirmados_40_49_f', 'confirmados_40_49_m', 'confirmados_50_59_f',
       'confirmados_50_59_m', 'confirmados_60_69_f', 'confirmados_60_69_m',
       'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f',
       'confirmados_80_plus_m', 'sintomas_tosse', 'sintomas_febre',
       'sintomas_dificuldade_respiratoria', 'sintomas_cefaleia',
       'sintomas_dores_musculares', 'sintomas_fraqueza_generalizada',
       'confirmados_f', 'confirmados_m', 'obitos_0_9_f', 'obitos_0_9_m',
       'obitos_10_19_f', 'obitos_10_19_m', 'obitos_20_29_f', 'obitos_20_29_m',
       'obitos_30_39_f', 'obitos_30_39_m', 'obit

### Redefinindo algumas features utilizando a primeira diferença
* O objetivo é transformar os valores que representam o quantitativo acumulado diariamente no quantitativo diário

In [148]:
# Fazendo cópia do Dataset orginal e após aplicação do filtro
df_pt_diff = df_pt.copy()
df_pt_diff.shape

(722, 63)

A coluna 'confirmados_0_9_m' possui uma lacuna em uma data crucial para os resultados por isso, está sendo feito o tratamento do NAN nesta coluna antes da aplicação da primeira diferença. A lacuna é para o registro 424 correspondente a data 2021-04-25

In [149]:
df_pt_diff[df_pt_diff['confirmados_0_9_m'].isna()]

,data,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,confirmados_0_9_f,confirmados_0_9_m,confirmados_10_19_f,confirmados_10_19_m,confirmados_20_29_f,confirmados_20_29_m,confirmados_30_39_f,confirmados_30_39_m,confirmados_40_49_f,confirmados_40_49_m,confirmados_50_59_f,confirmados_50_59_m,confirmados_60_69_f,confirmados_60_69_m,confirmados_70_79_f,confirmados_70_79_m,confirmados_80_plus_f,confirmados_80_plus_m,sintomas_tosse,sintomas_febre,sintomas_dificuldade_respiratoria,sintomas_cefaleia,sintomas_dores_musculares,sintomas_fraqueza_generalizada,confirmados_f,confirmados_m,obitos_0_9_f,obitos_0_9_m,obitos_10_19_f,obitos_10_19_m,obitos_20_29_f,obitos_20_29_m,obitos_30_39_f,obitos_30_39_m,obitos_40_49_f,obitos_40_49_m,obitos_50_59_f,obitos_50_59_m,obitos_60_69_f,obitos_60_69_m,obitos_70_79_f,obitos_70_79_m,obitos_80_plus_f,obitos_80_plus_m,obitos_f,obitos_m,ativos,internados_enfermaria,incidencia_nacional,incidencia_continente,rt_nacional,rt_continente
0,2020-02-26,0,0,0,0,NaN,NaN,NaN,25.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-27,0,0,0,0,NaN,NaN,NaN,51.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-28,0,0,0,0,NaN,NaN,NaN,59.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-29,0,0,0,0,NaN,NaN,NaN,70.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-03,0,0,0,0,NaN,NaN,NaN,85.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-02-03,2,2,0,0,NaN,NaN,NaN,85.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,NaN,NaN,NaN,NaN
424,2021-04-25,834442,478,792685,16965,348.00,98.00,NaN,NaN,24313.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455613.00,378523.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8056.00,8909.00,24792.00,250.00,72.10,68.30,0.98,0.99


In [150]:
df_pt_diff.dropna(subset=['confirmados_0_9_m'], axis=0,inplace=True)
df_pt_diff.reset_index(inplace=True, drop=True)

In [151]:
df_pt_diff.shape

(715, 63)

In [152]:
# Cálculo da primeira diferença

features_list_diff = ['recuperados','obitos','suspeitos','n_confirmados','confirmados_0_9_f', 'confirmados_0_9_m', 'confirmados_10_19_f',
                      'confirmados_10_19_m', 'confirmados_20_29_f', 'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m',
                      'confirmados_40_49_f', 'confirmados_40_49_m', 'confirmados_50_59_f', 'confirmados_50_59_m', 'confirmados_60_69_f',
                      'confirmados_60_69_m', 'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f', 'confirmados_80_plus_m',
                      'confirmados_f', 'confirmados_m', 'obitos_f', 'obitos_m'
                    ]

for col in features_list_diff:
  temp = df_pt_diff.loc[0,col]
  df_pt_diff[col] = df_pt_diff[col].diff()
  df_pt_diff.loc[0,col] = temp

In [153]:
# Verificação da existência de valores negativos após o cálculo da primeira diferença.
# Isso pode ocorrer devido aos erros de preenchimento do dataset

neg_cols = []

for col in features_list_diff:
  neg_check = df_pt_diff[df_pt_diff[col] < 0][col].count()

  if neg_check > 0:
    neg_cols.append(col)
    print(col,'--->',neg_check)
print('\nLista de colunas com valores negativos: \n', neg_cols)

confirmados_0_9_f ---> 8
confirmados_0_9_m ---> 6
confirmados_10_19_f ---> 3
confirmados_10_19_m ---> 4
confirmados_20_29_m ---> 2
confirmados_30_39_f ---> 1
confirmados_30_39_m ---> 1
confirmados_40_49_f ---> 2
confirmados_60_69_f ---> 1
confirmados_60_69_m ---> 2
confirmados_70_79_f ---> 3
confirmados_70_79_m ---> 3
confirmados_80_plus_f ---> 1
confirmados_80_plus_m ---> 3

Lista de colunas com valores negativos: 
 ['confirmados_0_9_f', 'confirmados_0_9_m', 'confirmados_10_19_f', 'confirmados_10_19_m', 'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m', 'confirmados_40_49_f', 'confirmados_60_69_f', 'confirmados_60_69_m', 'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f', 'confirmados_80_plus_m']


In [154]:
# Substituindo registros negativos pela mediana

print('Valores substituídos e mediana das colunas: \n')

for col in neg_cols:
  median_col = df_pt_diff[df_pt_diff[col] > 0][col].median()

  subs_list = df_pt_diff[df_pt_diff[col] < 0][col].to_list()
  print(col, '---> ', subs_list, '---> mediana: ', median_col)

  df_pt_diff[col] = df_pt_diff[col].replace(to_replace=subs_list, value=median_col)

Valores substituídos e mediana das colunas: 

confirmados_0_9_f --->  [-1.0, -5.0, -1.0, -2.0, -1.0, -3.0, -1.0, -9.0] ---> mediana:  40.0
confirmados_0_9_m --->  [-5.0, -6.0, -1.0, -4.0, -5.0, -7.0] ---> mediana:  42.0
confirmados_10_19_f --->  [-1.0, -6.0, -2.0] ---> mediana:  46.0
confirmados_10_19_m --->  [-2.0, -4.0, -3.0, -5.0] ---> mediana:  50.5
confirmados_20_29_m --->  [-118.0, -5.0] ---> mediana:  80.0
confirmados_30_39_f --->  [-5.0] ---> mediana:  71.0
confirmados_30_39_m --->  [-192.0] ---> mediana:  67.0
confirmados_40_49_f --->  [-2.0, -4.0] ---> mediana:  76.0
confirmados_60_69_f --->  [-14.0] ---> mediana:  47.0
confirmados_60_69_m --->  [-6.0, -9.0] ---> mediana:  39.0
confirmados_70_79_f --->  [-9.0, -1.0, -2.0] ---> mediana:  31.0
confirmados_70_79_m --->  [-5.0, -1.0, -4.0] ---> mediana:  27.0
confirmados_80_plus_f --->  [-1.0] ---> mediana:  36.5
confirmados_80_plus_m --->  [-2.0, -1.0, -1.0] ---> mediana:  20.0


### Criando novas features baseadas na coluna data

In [155]:
# Criando novas features baseadas na coluna 'date'
df_pt_diff['dia'] = df_pt_diff['data'].dt.day
df_pt_diff['mes'] = df_pt_diff['data'].dt.month
#df_pt_diff['ano'] = df_pt_diff['data'].dt.year
df_pt_diff['dia_da_semana'] = df_pt_diff['data'].dt.dayofweek

In [156]:
df_pt_diff.columns

Index(['data', 'confirmados', 'confirmados_novos', 'recuperados', 'obitos',
       'internados', 'internados_uci', 'lab', 'suspeitos', 'vigilancia',
       'n_confirmados', 'confirmados_0_9_f', 'confirmados_0_9_m',
       'confirmados_10_19_f', 'confirmados_10_19_m', 'confirmados_20_29_f',
       'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m',
       'confirmados_40_49_f', 'confirmados_40_49_m', 'confirmados_50_59_f',
       'confirmados_50_59_m', 'confirmados_60_69_f', 'confirmados_60_69_m',
       'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f',
       'confirmados_80_plus_m', 'sintomas_tosse', 'sintomas_febre',
       'sintomas_dificuldade_respiratoria', 'sintomas_cefaleia',
       'sintomas_dores_musculares', 'sintomas_fraqueza_generalizada',
       'confirmados_f', 'confirmados_m', 'obitos_0_9_f', 'obitos_0_9_m',
       'obitos_10_19_f', 'obitos_10_19_m', 'obitos_20_29_f', 'obitos_20_29_m',
       'obitos_30_39_f', 'obitos_30_39_m', 'obit

In [157]:
# Reordenando as colunas
new_order = [ 'data', 'mes','dia', 'dia_da_semana', 'confirmados', 'confirmados_novos', 'recuperados', 'obitos',
              'internados', 'internados_uci', 'lab', 'suspeitos', 'vigilancia',
              'n_confirmados', 'confirmados_0_9_f', 'confirmados_0_9_m',
              'confirmados_10_19_f', 'confirmados_10_19_m', 'confirmados_20_29_f',
              'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m',
              'confirmados_40_49_f', 'confirmados_40_49_m', 'confirmados_50_59_f',
              'confirmados_50_59_m', 'confirmados_60_69_f', 'confirmados_60_69_m',
              'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f',
              'confirmados_80_plus_m', 'sintomas_tosse', 'sintomas_febre',
              'sintomas_dificuldade_respiratoria', 'sintomas_cefaleia',
              'sintomas_dores_musculares', 'sintomas_fraqueza_generalizada',
              'confirmados_f', 'confirmados_m', 'obitos_0_9_f', 'obitos_0_9_m',
              'obitos_10_19_f', 'obitos_10_19_m', 'obitos_20_29_f', 'obitos_20_29_m',
              'obitos_30_39_f', 'obitos_30_39_m', 'obitos_40_49_f', 'obitos_40_49_m',
              'obitos_50_59_f', 'obitos_50_59_m', 'obitos_60_69_f', 'obitos_60_69_m',
              'obitos_70_79_f', 'obitos_70_79_m', 'obitos_80_plus_f',
              'obitos_80_plus_m', 'obitos_f', 'obitos_m', 'ativos',
              'internados_enfermaria', 'incidencia_nacional', 'incidencia_continente',
              'rt_nacional', 'rt_continente',]

df_pt_diff = df_pt_diff[new_order]

### Definindo coluna 'data' como index

In [158]:
df_pt_diff.set_index(df_pt_diff['data'], inplace=True)
df_pt_diff = df_pt_diff.drop(['data'], axis=1)
df_pt_diff = df_pt_diff.sort_index()
df_pt_diff.index

DatetimeIndex(['2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11',
               '2020-01-12', '2020-02-04',
               ...
               '2022-08-01', '2022-08-02', '2022-09-01', '2022-09-02',
               '2022-10-01', '2022-10-02', '2022-11-01', '2022-11-02',
               '2022-12-01', '2022-12-02'],
              dtype='datetime64[ns]', name='data', length=715, freq=None)

## Definindo intervalo que será utilizado para filtrar o dataset - 2021-03-15 a 2022-01-31

Definição dos Conjuntos de Treino e Teste:
* Treino: 2021-03-15 a 2022-01-21
* Teste: 2022-01-22 a 2022-01-31 (Intervalo de 10 dias para o conjunto de teste)
<br>
<br>Obs1.: Erro na data a partir de fev. 2022
<br>Obs2.: Iniciando em de março de 2021 pois a features 'incidencia_nacional', 'incidencia_continente', 'rt_nacional' e 'rt_continente' só passaram a ser contabilizadas a partir dessa data.

In [159]:
df_train = df_pt_diff.loc['2021-03-15':'2022-01-21', :].copy()
df_train = df_train.sort_index() # Reordena o dataset através do index depois de ter feito a filtragem

df_test = df_pt_diff.loc['2022-01-22':'2022-01-31', :].copy()
df_test = df_test.sort_index()

In [160]:
# Verifica se as datas estão corretamente ordenadas
df_train.head()

,mes,dia,dia_da_semana,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,confirmados_0_9_f,confirmados_0_9_m,confirmados_10_19_f,confirmados_10_19_m,confirmados_20_29_f,confirmados_20_29_m,confirmados_30_39_f,confirmados_30_39_m,confirmados_40_49_f,confirmados_40_49_m,confirmados_50_59_f,confirmados_50_59_m,confirmados_60_69_f,confirmados_60_69_m,confirmados_70_79_f,confirmados_70_79_m,confirmados_80_plus_f,confirmados_80_plus_m,sintomas_tosse,sintomas_febre,sintomas_dificuldade_respiratoria,sintomas_cefaleia,sintomas_dores_musculares,sintomas_fraqueza_generalizada,confirmados_f,confirmados_m,obitos_0_9_f,obitos_0_9_m,obitos_10_19_f,obitos_10_19_m,obitos_20_29_f,obitos_20_29_m,obitos_30_39_f,obitos_30_39_m,obitos_40_49_f,obitos_40_49_m,obitos_50_59_f,obitos_50_59_m,obitos_60_69_f,obitos_60_69_m,obitos_70_79_f,obitos_70_79_m,obitos_80_plus_f,obitos_80_plus_m,obitos_f,obitos_m,ativos,internados_enfermaria,incidencia_nacional,incidencia_continente,rt_nacional,rt_continente
data,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-15,3,15,0,814513,256,2371.00,10.00,996.00,231.00,NaN,NaN,16685.00,NaN,40.00,4.00,13.00,50.50,11.00,21.00,21.00,0.00,11.00,25.00,28.00,25.00,30.00,10.00,13.00,17.00,22.00,13.00,NaN,NaN,NaN,NaN,NaN,NaN,144.00,113.00,1.00,1.00,1.00,1.00,5.00,7.00,20.00,21.00,61.00,89.00,126.00,323.00,452.00,1026.00,1307.00,2226.00,5968.00,5059.00,5.00,5.00,36031.00,765.00,96.00,84.20,0.83,0.79
2021-03-16,3,16,1,814897,384,1173.00,13.00,955.00,213.00,NaN,NaN,15774.00,NaN,12.00,2.00,12.00,12.00,9.00,39.00,28.00,24.00,24.00,31.00,37.00,21.00,27.00,26.00,17.00,13.00,27.00,21.00,NaN,NaN,NaN,NaN,NaN,NaN,193.00,189.00,1.00,1.00,1.00,1.00,5.00,7.00,20.00,21.00,61.00,90.00,128.00,323.00,452.00,1027.00,1309.00,2229.00,5970.00,5061.00,6.00,7.00,35229.00,742.00,96.00,82.30,0.83,0.79
2021-03-17,3,17,2,815570,673,1058.00,15.00,856.00,205.00,NaN,NaN,15183.00,NaN,12.00,12.00,19.00,32.00,56.00,37.00,50.00,42.00,60.00,46.00,72.00,44.00,40.00,41.00,29.00,22.00,40.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,378.00,296.00,1.00,1.00,1.00,1.00,5.00,7.00,20.00,21.00,61.00,90.00,128.00,324.00,453.00,1028.00,1311.00,2232.00,5973.00,5065.00,6.00,9.00,34829.00,651.00,90.30,79.10,0.84,0.80
2021-03-18,3,18,3,816055,485,580.00,21.00,828.00,187.00,NaN,NaN,15268.00,NaN,6.00,15.00,12.00,15.00,37.00,43.00,27.00,44.00,40.00,37.00,42.00,30.00,28.00,28.00,31.00,15.00,16.00,14.00,NaN,NaN,NaN,NaN,NaN,NaN,239.00,241.00,1.00,1.00,1.00,1.00,5.00,7.00,20.00,21.00,61.00,90.00,129.00,325.00,453.00,1031.00,1313.00,2237.00,5976.00,5071.00,6.00,15.00,34713.00,641.00,90.30,79.10,0.84,0.80
2021-03-19,3,19,4,816623,568,1571.00,11.00,789.00,182.00,NaN,NaN,14915.00,NaN,18.00,17.00,22.00,24.00,45.00,44.00,36.00,52.00,53.00,37.00,37.00,40.00,49.00,23.00,10.00,22.00,33.00,5.00,NaN,NaN,NaN,NaN,NaN,NaN,303.00,264.00,1.00,1.00,1.00,1.00,5.00,7.00,20.00,21.00,61.00,91.00,129.00,325.00,453.00,1035.00,1314.00,2238.00,5978.00,5073.00,3.00,8.00,33699.00,607.00,87.20,75.70,0.86,0.84


In [161]:
print('Dimensões do conjunto de treino: ', df_train.shape)
print('Dimensões do conjunto de teste: ', df_test.shape)

Dimensões do conjunto de treino:  (302, 65)
Dimensões do conjunto de teste:  (10, 65)


In [162]:
df_train.columns

Index(['mes', 'dia', 'dia_da_semana', 'confirmados', 'confirmados_novos',
       'recuperados', 'obitos', 'internados', 'internados_uci', 'lab',
       'suspeitos', 'vigilancia', 'n_confirmados', 'confirmados_0_9_f',
       'confirmados_0_9_m', 'confirmados_10_19_f', 'confirmados_10_19_m',
       'confirmados_20_29_f', 'confirmados_20_29_m', 'confirmados_30_39_f',
       'confirmados_30_39_m', 'confirmados_40_49_f', 'confirmados_40_49_m',
       'confirmados_50_59_f', 'confirmados_50_59_m', 'confirmados_60_69_f',
       'confirmados_60_69_m', 'confirmados_70_79_f', 'confirmados_70_79_m',
       'confirmados_80_plus_f', 'confirmados_80_plus_m', 'sintomas_tosse',
       'sintomas_febre', 'sintomas_dificuldade_respiratoria',
       'sintomas_cefaleia', 'sintomas_dores_musculares',
       'sintomas_fraqueza_generalizada', 'confirmados_f', 'confirmados_m',
       'obitos_0_9_f', 'obitos_0_9_m', 'obitos_10_19_f', 'obitos_10_19_m',
       'obitos_20_29_f', 'obitos_20_29_m', 'obitos_30_39_f', 

In [163]:
# Listando colunas nulas após definição do período
null_cols = []

for col in df_train.columns:
  if df_train[col].sum() == 0:
    null_cols.append(col)

print(null_cols)

['lab', 'suspeitos', 'n_confirmados', 'sintomas_tosse', 'sintomas_febre', 'sintomas_dificuldade_respiratoria', 'sintomas_cefaleia', 'sintomas_dores_musculares', 'sintomas_fraqueza_generalizada']


In [164]:
# Excluindo colunas nulas
df_train = df_train.drop(null_cols, axis=1)
df_test = df_test.drop(null_cols, axis=1)

In [165]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 302 entries, 2021-03-15 to 2022-01-21
Data columns (total 56 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mes                    302 non-null    int64  
 1   dia                    302 non-null    int64  
 2   dia_da_semana          302 non-null    int64  
 3   confirmados            302 non-null    int64  
 4   confirmados_novos      302 non-null    int64  
 5   recuperados            302 non-null    float64
 6   obitos                 302 non-null    float64
 7   internados             302 non-null    float64
 8   internados_uci         302 non-null    float64
 9   vigilancia             302 non-null    float64
 10  confirmados_0_9_f      302 non-null    float64
 11  confirmados_0_9_m      302 non-null    float64
 12  confirmados_10_19_f    302 non-null    float64
 13  confirmados_10_19_m    302 non-null    float64
 14  confirmados_20_29_f    302 non-null    

## Correlação

In [166]:
round(df_train.corr(),4)

,mes,dia,dia_da_semana,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,vigilancia,confirmados_0_9_f,confirmados_0_9_m,confirmados_10_19_f,confirmados_10_19_m,confirmados_20_29_f,confirmados_20_29_m,confirmados_30_39_f,confirmados_30_39_m,confirmados_40_49_f,confirmados_40_49_m,confirmados_50_59_f,confirmados_50_59_m,confirmados_60_69_f,confirmados_60_69_m,confirmados_70_79_f,confirmados_70_79_m,confirmados_80_plus_f,confirmados_80_plus_m,confirmados_f,confirmados_m,obitos_0_9_f,obitos_0_9_m,obitos_10_19_f,obitos_10_19_m,obitos_20_29_f,obitos_20_29_m,obitos_30_39_f,obitos_30_39_m,obitos_40_49_f,obitos_40_49_m,obitos_50_59_f,obitos_50_59_m,obitos_60_69_f,obitos_60_69_m,obitos_70_79_f,obitos_70_79_m,obitos_80_plus_f,obitos_80_plus_m,obitos_f,obitos_m,ativos,internados_enfermaria,incidencia_nacional,incidencia_continente,rt_nacional,rt_continente
mes,1.00,-0.05,-0.01,-0.00,-0.25,-0.28,0.01,-0.02,-0.00,-0.17,-0.30,-0.30,-0.28,-0.28,-0.21,-0.17,-0.26,-0.23,-0.26,-0.25,-0.23,-0.22,-0.20,-0.18,-0.21,-0.22,-0.21,-0.23,-0.25,-0.25,NaN,0.37,0.02,NaN,-0.31,0.05,-0.02,0.19,0.14,0.26,0.21,0.12,0.13,0.10,0.12,0.08,0.09,0.08,0.00,0.02,-0.25,-0.03,-0.32,-0.31,0.12,0.13
dia,-0.05,1.00,-0.04,0.21,0.00,-0.11,-0.40,-0.40,-0.43,-0.12,0.02,0.01,0.02,0.01,0.03,0.05,0.01,0.02,0.00,0.01,-0.01,-0.00,-0.04,-0.04,-0.09,-0.09,-0.24,-0.20,-0.00,0.01,NaN,0.27,0.13,NaN,0.25,0.36,0.37,0.38,0.39,0.39,0.40,0.41,0.41,0.41,0.41,0.41,0.39,0.40,-0.39,-0.41,-0.12,-0.39,-0.04,-0.04,0.08,0.07
dia_da_semana,-0.01,-0.04,1.00,0.05,0.06,0.02,0.02,0.00,-0.00,0.05,0.04,0.04,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.05,0.05,0.05,0.06,0.06,0.07,0.07,0.08,0.07,0.06,0.06,NaN,0.02,0.04,NaN,0.02,0.03,0.01,0.02,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.02,0.06,0.01,0.05,0.05,0.01,0.01
confirmados,-0.00,0.21,0.05,1.00,0.74,0.68,-0.15,-0.10,-0.29,0.67,0.77,0.76,0.77,0.77,0.72,0.72,0.75,0.74,0.75,0.75,0.70,0.70,0.68,0.67,0.65,0.66,0.38,0.46,0.74,0.75,NaN,0.71,0.69,NaN,0.81,0.82,0.48,0.77,0.76,0.79,0.79,0.74,0.70,0.69,0.72,0.67,0.64,0.65,-0.15,-0.16,0.72,-0.07,0.89,0.89,0.31,0.30
confirmados_novos,-0.25,0.00,0.06,0.74,1.00,0.88,0.21,0.29,0.12,0.89,0.94,0.94,0.99,0.99,0.98,0.97,1.00,1.00,1.00,1.00,0.98,0.98,0.98,0.97,0.96,0.96,0.82,0.87,1.00,1.00,NaN,0.18,0.73,NaN,0.59,0.33,0.01,0.24,0.24,0.23,0.23,0.20,0.16,0.16,0.19,0.15,0.11,0.12,0.22,0.20,0.93,0.31,0.94,0.94,0.37,0.36
recuperados,-0.28,-0.11,0.02,0.68,0.88,1.00,0.38,0.45,0.30,0.90,0.85,0.85,0.88,0.88,0.84,0.81,0.88,0.86,0.88,0.88,0.86,0.85,0.87,0.86,0.89,0.89,0.80,0.85,0.88,0.88,NaN,0.12,0.60,NaN,0.55,0.28,-0.01,0.18,0.17,0.15,0.15,0.12,0.09,0.09,0.12,0.08,0.07,0.08,0.39,0.37,0.93,0.48,0.96,0.96,0.26,0.25
obitos,0.01,-0.40,0.02,-0.15,0.21,0.38,1.00,0.98,0.94,0.48,0.13,0.13,0.16,0.16,0.18,0.17,0.19,0.18,0.20,0.20,0.25,0.24,0.31,0.32,0.39,0.37,0.62,0.56,0.22,0.20,NaN,-0.33,0.05,NaN,-0.40,-0.46,-0.61,-0.49,-0.53,-0.48,-0.53,-0.59,-0.60,-0.60,-0.57,-0.61,-0.58,-0.58,0.99,0.99,0.41,0.98,0.81,0.81,0.21,0.19
internados,-0.02,-0.40,0.00,-0.10,0.29,0.45,0.98,1.00,0.97,0.56,0.21,0.21,0.24,0.24,0.26,0.25,0.26,0.26,0.27,0.27,0.32,0.31,0.38,0.39,0.45,0.44,0.66,0.62,0.29,0.28,NaN,-0.33,0.12,NaN,-0.34,-0.44,-0.60,-0.48,-0.52,-0.48,-0.52,-0.58,-0.59,-0.59,-0.56,-0.60,-0.57,-0.57,0.97,0.98,0.49,1.00,0.84,0.85,0.27,0.25
internados_uci,-0.00,-0.43,-0.00,-0.29,0.12,0.30,0.94,0.97,1.00,0.40,0.03,0.03,0.08,0.07,0.11,0.10,0.10,0.10,0.10,0.11,0.16,0.15,0.21,0.23,0.28,0.27,0.53,0.47,0.12,0.11,NaN,-0.43,-0.00,NaN,-0.47,-0.56,-0.62,-0.58,-0.62,-0.58,-0.63,-0.67,-0.67,-0.66,-0.64,-0.67,-0.63,-0.64,0.93,0.95,0.32,0.96,0.32,0.33,0.03,0.02
vigilancia,-0.17,-0.12,0.05,0.67,0.89,0.90,0.48,0.56,0.40,1.00,0.85,0.85,0.89,0.89,0.85,0.83,0.88,0.87,0.88,0.88,0.85,0.84,0.87,0.86,0.89,0.90,0.84,0.87,0.89,0.88,NaN,0.09,0.66,NaN,0.44,0.19,-0.11,0.15,0.10,0.11,0.11,0.06,0.02,0.03,0.06,0.01,-0.01,0.00,0.49,0.48,0.97,0.58,0.96,0.96,0.39,0.38


# Preparação dos dados para o Modelo de Machine Learning

## Seleção de Features e definição do Target

In [167]:
# Armazenando os nomes das colunas como lista

features_names = df_train.columns.to_list()

# Definindo a variável alvo e as features
target_name = 'internados_uci' #'recuperados' #'confirmados_novos' #'obitos' #'internados'

confirmados_list = ['confirmados_novos', 'confirmados', 'confirmados_0_9_f', 'confirmados_0_9_m',
                    'confirmados_10_19_f', 'confirmados_10_19_m', 'confirmados_20_29_f',
                    'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m',
                    'confirmados_40_49_f', 'confirmados_40_49_m', 'confirmados_50_59_f',
                    'confirmados_50_59_m', 'confirmados_60_69_f', 'confirmados_60_69_m',
                    'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f',
                    'confirmados_80_plus_m', 'confirmados_f', 'confirmados_m']

obitos_list = ['obitos', 'obitos_0_9_m', 'obitos_0_9_f', 'obitos_10_19_m', 'obitos_10_19_f', 'obitos_20_29_f', 'obitos_20_29_m',
               'obitos_30_39_f', 'obitos_30_39_m', 'obitos_40_49_f', 'obitos_40_49_m', 
               'obitos_50_59_f', 'obitos_50_59_m', 'obitos_60_69_f', 'obitos_60_69_m', 
               'obitos_70_79_f', 'obitos_70_79_m', 'obitos_80_plus_f', 'obitos_80_plus_m', 
               'obitos_f', 'obitos_m']

internados_list = ['internados', 'internados_uci', 'internados_enfermaria'] #+ obitos_list

if target_name == 'confirmados_novos':
  for name in confirmados_list:
    features_names.remove(name)
elif target_name == 'obitos':
  for name in obitos_list:
    features_names.remove(name)
elif target_name == 'internados':
  for name in internados_list:
    features_names.remove(name)
elif target_name == 'internados_uci':
  features_names.remove(target_name)
  features_names.remove('internados')
  #features_names.remove('internados_enfermaria')
else:
  features_names.remove(target_name)

print(features_names)

['mes', 'dia', 'dia_da_semana', 'confirmados', 'confirmados_novos', 'recuperados', 'obitos', 'vigilancia', 'confirmados_0_9_f', 'confirmados_0_9_m', 'confirmados_10_19_f', 'confirmados_10_19_m', 'confirmados_20_29_f', 'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m', 'confirmados_40_49_f', 'confirmados_40_49_m', 'confirmados_50_59_f', 'confirmados_50_59_m', 'confirmados_60_69_f', 'confirmados_60_69_m', 'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f', 'confirmados_80_plus_m', 'confirmados_f', 'confirmados_m', 'obitos_0_9_f', 'obitos_0_9_m', 'obitos_10_19_f', 'obitos_10_19_m', 'obitos_20_29_f', 'obitos_20_29_m', 'obitos_30_39_f', 'obitos_30_39_m', 'obitos_40_49_f', 'obitos_40_49_m', 'obitos_50_59_f', 'obitos_50_59_m', 'obitos_60_69_f', 'obitos_60_69_m', 'obitos_70_79_f', 'obitos_70_79_m', 'obitos_80_plus_f', 'obitos_80_plus_m', 'obitos_f', 'obitos_m', 'ativos', 'internados_enfermaria', 'incidencia_nacional', 'incidencia_continente', 'rt_nacional',

In [168]:
y_train = df_train[[target_name]]
y_test = df_test[[target_name]]

X_train = df_train[features_names]
X_test = df_test[features_names]

In [169]:
#X_train.info()

In [170]:
# Descrição das features selecionadas
pd.options.display.float_format = '{:.2f}'.format

X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
mes,302.00,7.48,2.99,1.00,5.00,8.00,10.00,12.00
dia,302.00,16.10,8.75,1.00,9.00,16.00,23.00,31.00
dia_da_semana,302.00,2.99,1.99,0.00,1.00,3.00,5.00,6.00
confirmados,302.00,993315.18,252809.67,431623.00,839621.50,964743.50,1083418.50,2690690.00
confirmados_novos,302.00,3680.20,8469.54,158.00,598.00,1182.50,2886.25,58530.00
recuperados,302.00,2766.99,5431.81,226.00,605.00,1088.50,2631.00,44610.00
obitos,302.00,17.95,37.16,0.00,4.00,8.00,14.00,258.00
vigilancia,302.00,59468.30,68915.38,14734.00,22031.50,34922.00,74863.00,639307.00
confirmados_0_9_f,302.00,190.79,588.26,0.00,22.25,56.00,136.00,4768.00
confirmados_0_9_m,302.00,199.96,623.92,0.00,24.25,59.50,130.75,5116.00


In [171]:
# Descrição da variável alvo
y_train.describe().T

,count,mean,std,min,25%,50%,75%,max
internados_uci,302.00,155.95,154.39,49.00,75.00,119.00,156.50,904.00


In [172]:
# Pipeline para preparação das variáveis numéricas

numeric_pipeline = Pipeline(steps=[
                                   ('imputer', SimpleImputer(strategy='median')), # fill_value=0
                                   #('normalization', MinMaxScaler())
])

In [173]:
# Ajustando variáveis numéricas usando o pipeline

X_train_prepared = numeric_pipeline.fit_transform(X_train)
y_train_prepared = numeric_pipeline.fit_transform(y_train)

X_test_prepared = numeric_pipeline.fit_transform(X_test)
y_test_prepared = numeric_pipeline.fit_transform(y_test)

# Implementação do XGBoost

### Definição da seed e Instanciando o XGBoost

In [174]:
seed = 1275

xgb = XGBRegressor( #tree_method = 'gpu_hist',
                    booster='gbtree',
                    objective='reg:squarederror', max_depth=15,
                    learning_rate=0.1, n_estimators=100,
                    random_state=seed, n_jobs=-1
)

### Time Series cross-validator

In [175]:
tscv = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None)
print(tscv)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None)


### Avaliação do Modelo antes da busca pelos melhores parâmetros

In [176]:
# Avaliação do modelo antes da busca de parâmetros

def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error", "neg_mean_squared_error", "neg_mean_absolute_percentage_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    mape = -cv_results['test_neg_mean_absolute_percentage_error']
    mse = -cv_results['test_neg_mean_squared_error']
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    
    print(
        f"Mean Absolute Error (MAE):     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Mean Absolute Percentage Error (MAPE): {mape.mean():.3f} +/- {mape.std():.3f}\n"
        f"Mean Squared Error (MSE): {mse.mean():.3f} +/- {mse.std():.3f}\n"
        f"Root Mean Squared Error (RMSE): {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )
    #print(cv_results.keys())

evaluate(xgb, X_train_prepared, y_train_prepared, cv=tscv)

Mean Absolute Error (MAE):     14.412 +/- 7.763
Mean Absolute Percentage Error (MAPE): 0.106 +/- 0.049
Mean Squared Error (MSE): 1301.705 +/- 2080.481
Root Mean Squared Error (RMSE): 28.320 +/- 22.354


### Busca dos Melhores Parâmetros

In [177]:
param_distributions = [
                        {
                          'n_estimators': [100, 120, 130, 140, 150, 160, 200, 225, 235, 250, 265, 275], 
                          'learning_rate':[0.01, 0.03, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
                          'max_depth':[4, 5, 6, 7, 8, 9, 10, 12, 14],
                          'booster':['gbtree', 'gblinear'],
                          'objective':['reg:squarederror'], #,'reg:logistic'],
                          'gamma':[0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0],
                          #'tree_method': ['gpu_hist']
                        }
                      ]

In [178]:
rnd_search = RandomizedSearchCV(estimator=xgb, 
                                param_distributions = param_distributions, 
                                n_iter=50, scoring='neg_mean_squared_error', #'neg_root_mean_squared_error', #'neg_mean_absolute_percentage_error'
                                n_jobs=-1, cv=tscv, random_state=seed
                              )
rnd_search.fit(X_train_prepared, y_train_prepared)

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                   estimator=XGBRegressor(max_depth=15, n_jobs=-1,
                                          objective='reg:squarederror',
                                          random_state=1275),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'booster': ['gbtree', 'gblinear'],
                                         'gamma': [0, 0.1, 0.5, 1.0, 1.5, 2.0,
                                                   2.5, 3.0],
                                         'learning_rate': [0.01, 0.03, 0.05,
                                                           0.1, 0.2, 0.3, 0.4,
                                                           0.5, 0.6, 0.7, 0.8],
                                         'max_depth': [4, 5, 6, 7, 8, 9, 10, 12,
                                                       14],
                                         'n_estimators': [100, 120, 

In [179]:
print('Melhores parâmetros: ', rnd_search.best_params_)

Melhores parâmetros:  {'objective': 'reg:squarederror', 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.03, 'gamma': 0, 'booster': 'gbtree'}


### Avaliação do Modelo após a Busca pelos Melhores Parâmetros

In [180]:
# Avaliação do modelo após a busca por parâmetros

xgb = XGBRegressor(**rnd_search.best_params_, random_state=seed)
print(xgb.get_params)
print('\n')

evaluate(xgb, X_train_prepared, y_train_prepared, cv=tscv)

<bound method XGBModel.get_params of XGBRegressor(learning_rate=0.03, max_depth=10, n_estimators=200,
             objective='reg:squarederror', random_state=1275)>


Mean Absolute Error (MAE):     14.672 +/- 8.010
Mean Absolute Percentage Error (MAPE): 0.107 +/- 0.050
Mean Squared Error (MSE): 1233.808 +/- 2039.850
Root Mean Squared Error (RMSE): 27.920 +/- 21.314


## Testando o Modelo

In [181]:
xgb = XGBRegressor(**rnd_search.best_params_, random_state=seed)
xgb.fit(X_train_prepared,y_train_prepared)
y_predicted = xgb.predict(X_test_prepared)

y_predicted

array([156.29758, 159.949  , 159.62973, 156.09949, 152.22345, 156.90675,
       153.06445, 153.06445, 154.87793, 155.11096], dtype=float32)

In [182]:
# Comparação dos resultados

print('(Teste, Previsão) --- Previsão-Teste')

for pair in zip(np.reshape(y_test_prepared, len(y_test_prepared)), np.round(y_predicted,0)):
  print(pair, '---', pair[1]-pair[0])

(Teste, Previsão) --- Previsão-Teste
(154.0, 156.0) --- 2.0
(160.0, 160.0) --- 0.0
(172.0, 160.0) --- -12.0
(158.0, 156.0) --- -2.0
(154.0, 152.0) --- -2.0
(147.0, 157.0) --- 10.0
(152.0, 153.0) --- 1.0
(153.0, 153.0) --- 0.0
(160.0, 155.0) --- -5.0
(160.0, 155.0) --- -5.0


In [183]:
# Métricas para o conjunto de testes
def test_metrics(y_pred, y_test):
  mae = mean_absolute_error(y_pred, y_test)
  mape = mean_absolute_percentage_error(y_pred, y_test)
  mse = mean_squared_error(y_pred, y_test)
  rmse = np.sqrt(mse)
      
  print(
        f"Mean Absolute Error (MAE):     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Mean Absolute Percentage Error (MAPE): {mape.mean():.3f} +/- {mape.std():.3f}\n"
        f"Mean Squared Error (MSE): {mse.mean():.3f} +/- {mse.std():.3f}\n"
        f"Root Mean Squared Error (RMSE): {rmse.mean():.3f} +/- {rmse.std():.3f}"
       )
  
test_metrics(y_predicted, y_test_prepared)

Mean Absolute Error (MAE):     3.944 +/- 0.000
Mean Absolute Percentage Error (MAPE): 0.025 +/- 0.000
Mean Squared Error (MSE): 31.449 +/- 0.000
Root Mean Squared Error (RMSE): 5.608 +/- 0.000
